# Ejercicio 1: 

Escribe un programa en Python que use Biopython para hacer una búsqueda BLAST de una secuencia de ADN que introduzcas por teclado. 
El programa debe mostrar por pantalla el número de resultados obtenidos, el E-value del mejor resultado y la descripción de la secuencia más similar. 
Hágalo de forma online y local.

In [2]:
import warnings
import os
import pandas as pd

In [3]:
warnings.filterwarnings("ignore")

from debug_sequence.functions import check_sequence
from Bio.Blast import NCBIWWW, NCBIXML
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Blast import NCBIXML

### 1.1. Instalación de BLAST

1. Se descarga la versión para el equipo de BLAST desde el siguiente enlace: [BLAST](https://ftp.ncbi.nlm.nih.gov/blast/executables/blast+/LATEST/)
2. Se ejecuta, en nuestro caso, el *.exe* descargado y se instala en el equipo.
3. Se configura el *path* de BLAST como variable de entorno del sistema.
4. Se verifica la instalación con el comando `blastn --version`.

### 1.2. Secuencia de ADN por teclado

In [4]:
def get_sequence():
    sequence = input("Introduce la secuencia de ADN: ").strip()
    sequence = check_sequence(sequence)
    return sequence

### 1.3. Implementación online

In [5]:
def search_online_sequence():
    sequence = get_sequence()

    result = NCBIWWW.qblast("blastn", "nt", sequence)
    blast_records = NCBIXML.read(result)
    
    if not blast_records.alignments:
        print("No se encontraron resultados.")
        
    num_results = len(blast_records.alignments)

    best_alignment = blast_records.alignments[0]
    best_hsp = best_alignment.hsps[0]
    e_value = best_hsp.expect
    description = best_alignment.title
    
    print("\nResultados BLAST:")
    print(f"Número de resultados: {num_results}")
    print(f"E-value del mejor resultado: {e_value}")
    print(f"Descripción de la secuencia más similar: {description}")

### 1.4. Implementación local

Para usar las bases de datos proporcionadas por el NCBI, es necesario descargarlas mediante el comando `perl update_blastdb.pl --decompress core_nt`. Para este ejercicio, se usará la base de datos `human_genome` y `core_nt`.

- **Base de datos `human_genome`**: Base de datos de ensamblaje actual del genoma humano refseq (GRCh). [Información disponible](https://www.ncbi.nlm.nih.gov/books/NBK62345/#blast_ftp_site.The_blastdb_subdirectory)
- **Base de datos `core_nt`**:  La base de datos core_nt es nt sin la mayoría de las secuencias de cromosomas eucariotas. La mayoría de las búsquedas BLAST de nucleótidos con core_nt serán similares a las de la base de datos nt. Sin embargo, core_nt es mejor que nt para alcanzar los objetivos de búsqueda BLAST más comunes, como la identificación de secuencias relacionadas con genes, como secuencias de transcripción y cromosomas bacterianos completos. Esto se debe a que, en los últimos años, nt ha adquirido más contenido de baja relevancia, no anotado y no génico. Para esta base de datos, debido a las limitaciones de recursos, se han descargado los archivos desde el 00 al 20. [Información disponible](https://bioinformaticsonline.com/news/view/44640/new-blast-core-nucleotide-database-core-nT)


In [6]:
def search_local_sequence():
    os.environ["CMD"] = input("Introduce la ruta del ejecutable de BLAST: ")
    os.environ["BLASTDB"] = input("Introduce la ruta de la base de datos de BLAST: ")
    
    sequence = get_sequence()

    fasta_file = "query.fasta"
    with open(fasta_file, "w") as f:
        f.write(">query\n")
        f.write(sequence)
    
    blastn_cline = NcbiblastnCommandline(
        cmd=os.environ["CMD"],
        query=fasta_file,
        db=os.environ["BLASTDB"],
        evalue=0.001,
        outfmt=5,
        out="results.xml"
    )

    blastn_cline()

    with open("results.xml") as result_handle:
        blast_records = NCBIXML.parse(result_handle)
        for blast_record in blast_records:
            if blast_record.alignments:
                print(f"Número de resultados: {len(blast_record.alignments)}")

                best_alignment = blast_record.alignments[0]
                best_hsp = best_alignment.hsps[0]
                print(f"E-value del mejor resultado: {best_hsp.expect}")
                print(f"Descripción del mejor resultado: {best_alignment.title}")
            else:
                print("No se encontraron coincidencias.")
    
    os.remove(fasta_file)
    os.remove("results.xml")

### 1.5. Comparación de la implementación local vs online

Debido a las limitaciones de recursos, se decide comparar los resultados entre las diversas bases de datos, puesto que la implementación online utiliza la base de datos completa de `nt` y la en implementación local, únicamente se dispone de un subconjunto de la base de datos `core_nt` y `human_genome`. Además, se comparará el tiempo de ejecución entre las implementaciones.

Para ello, se decide alinear mediante BLAST parte de una secuencia conocida que corresponde al gen de la hemoglobina:

...`TACTGCCCTGTGGGGCAAGGTGAACGTGGATGAAGTTGGTGGTGAGGCCCTGGGCAGGTTGGTATCAAGGTTACAAGACAGGTTGCTGGTGGTCTACCCTT`...

**CMD** 

C:\Users\Susana\Desktop\blast\blast-2.16.0+\bin\blastn.exe

**DB**

C:\Users\Susana\Desktop\blast\blast-2.16.0+\db\human_genome\GCF_000001405.39_top_level

C:\Users\Susana\Desktop\blast\blast-2.16.0+\db\nt\core_nt

In [7]:
import time

times_execution = {}

#### 1.5.1. Implementación online

In [8]:
start = time.time()
search_online_sequence()
end = time.time()

times_execution["online"] = end - start


Resultados BLAST:
Número de resultados: 50
E-value del mejor resultado: 5.3338e-33
Descripción de la secuencia más similar: gi|1584133852|gb|MK476408.1| Homo sapiens voucher Bantu_1_1 hemoglobin subunit beta (HBB) gene, complete cds


#### 1.5.2. Implementación local - `human_genome`

In [9]:
start = time.time()
search_local_sequence()
end = time.time()

times_execution["local human genome"] = end - start

Número de resultados: 1
E-value del mejor resultado: 2.24631e-36
Descripción del mejor resultado: gi|568815587|ref|NC_000011.10| Homo sapiens chromosome 11, GRCh38.p13 Primary Assembly


#### 1.5.3. Implementación local - subconjunto de `core_nt`

In [10]:
start = time.time()
search_local_sequence()
end = time.time()

times_execution["local subconjunto core_nt"] = end - start

Número de resultados: 500
E-value del mejor resultado: 1.59063e-34
Descripción del mejor resultado: gi|1584132808|gb|MK475886.1| Homo sapiens voucher Baka_79_1 hemoglobin subunit beta (HBB) gene, complete cds


#### 1.5.4. Comparación de resultados

Por un lado, la implementación local mediante la base de datos `core_nt` es la que más se asemeja a la online, la cual busca en la base de datos completa `nt`. Por otro lado, la base de datos `human_genome` es la que menos resultados arroja, debido a que es una base de datos específica del genoma humano.

In [11]:
df = pd.DataFrame.from_dict(times_execution, orient="index", columns=["Tiempo de ejecución (s)"])
display(df)

,Tiempo de ejecución (s)
online,67.487276
local human genome,19.800459
local subconjunto core_nt,524.341397


Sin embargo, la búsqueda local, debido a la falta de recursos, es la que más tarda. Por lo que las búsquedas locales se deben realizar en el caso de que se dispongan de los recursos necesarios.